<a href="https://colab.research.google.com/github/kz-yoshida/python/blob/branch1/2020_03_15_FpayForecast_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# module import
import pandas as pd

In [0]:
# GCP 認証
from google.colab import auth
auth.authenticate_user()

In [0]:
# Bigquery からData import
project_id = 'kouzoh-analytics-jp-prod'

df = pd.io.gbq.read_gbq(f'''

 With 
## ①本日から3/31までの日数を生成
    Days AS (    
        SELECT DISTINCT
            date
          , date_diff(date,Date_sub(CURRENT_DATE('Asia/Tokyo'),Interval 1 day), day)  AS Elapsed_Days  -- 前日から各レコード日までの日数 
          , CASE WHEN Extract(DAYOFWEEK from date) = 1 AND Extract(Month from date) = 3 THEN 1 ELSE 0 END AS Sunday_Flg  -- 3月の日曜はSunday対象 
        FROM
          UNNEST(GENERATE_DATE_ARRAY(
             CURRENT_DATE('Asia/Tokyo'), '2020-03-31'            
          )) AS date
), 
## ②対象となるMiddle以上の加盟店＆Fpay情報を抽出
TargetBrandList AS (
  SELECT
        MIN(Partner_Size) AS Partner_Size
      , MAX(mst.shops_category1) AS shops_category1 
      , CAST(mst.PartnerID AS STRING) AS PartnerID
      , MAX(mst.Partner_Name) AS Partner_Name
      , mst.BrandID
      , MAX(mst.Brand_nm) AS Brand_nm
      , MIN(mst.Deploy_Date) AS Deploy_Date
      ## 現在の数値
      , COUNT(mst_sh.seq ) AS Current_Cnt_Shops  -- 現在のSHOP数
      , SUM(fpay_flag)  AS Current_Cnt_FpayShops  -- 現在のFpaySHOP数
      , ROUND( SUM(fpay_flag) /COUNT(mst_sh.seq )  ,2) AS Current_FpayRate  -- 現在のFpay率
      , COUNT(mst_sh.seq ) - SUM(fpay_flag) AS Current_Cnt_Mipay  -- 現在の未FpaySHOP数
      ## 空き間口に対して平均Dayでどれくらいの率を埋めてきたかを算出
      , IFNULL(SAFE_DIVIDE(sum( CASE WHEN Date(mst.FirstpaymentAt,'Japan') >= '2020-01-01'THEN fpay_flag ELSE NULL END),              -- 3Q以降のFpay数 
        COUNT(mst_sh.seq )  - sum(CASE WHEN Date(mst.FirstpaymentAt,'Japan') <= '2019-12-31' THEN fpay_flag ELSE NULL END)), 0)   -- 現在のShop数 - ２QendのFpay数（=3Qの空き間口数）
        / date_diff(CURRENT_DATE('Asia/Tokyo'),'2020-01-01', day) 　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　-- 3Qの日数
        AS Velocity  　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 -- DAYの平均Fpay獲得速度（空き間口に対してDAYでどらくらいの割合を獲得するか）
      ## 2Qendの数値
      , COUNT(CASE WHEN Deploy_Date <= '2019-12-31' THEN mst_sh.seq ELSE 0 END) AS SecondQ_Cnt_Shops  -- 2QendのSHOP数
      , SUM( CASE WHEN Date(mst.FirstpaymentAt,'Japan') <= '2019-12-31' THEN fpay_flag ELSE 0 END) AS SecondQ_Cnt_FpayShops  -- 2QendのFpaySHOP数      
    FROM `mercari-analytics-jp.z_issei.191013_mst_partner_v01` mst
      join `merpay-tmp-exdata-jp.connect_data.t_merpay_shop_unigue` as mst_sh
      on (mst.ShopID = mst_sh.ShopID )
    WHERE
        Partner_Size != 'Small' 
    GROUP BY 
        PartnerID
      , BrandID
    UNION ALL
    SELECT
        *
    FROM `merpay-tmp-exdata-jp.connect_data.20200229_NewShops_Deploy` 
    )
    
    
## ③CPN踏まえた獲得速度の調整
    SELECT
        Days.*
      , TargetBrandList.*
      , F_CPN_Flag
      , D_CPN_Flag
      ## CPN期間も含めた全期間のDAYの獲得速度
    , CASE 
           WHEN Days.date < Deploy_Date THEN 0
           WHEN Sunday_Flg = 1 THEN Velocity +0.03 　-- CPN前は従来の獲得速度を踏襲
           WHEN date < CPN_StartDate THEN Velocity 　-- CPN前は従来の獲得速度を踏襲
           WHEN date BETWEEN CPN_StartDate AND CPN_EndDate THEN 
                CASE 
       ## FEVER CPN期間の獲得速度の調整
                     WHEN F_CPN_Flag = '3.提案中'　 THEN Velocity + 0.015   -- CPN実施で3%UP×確度50%    
                     WHEN F_CPN_Flag in ( "4.合意","完了")　THEN Velocity + 0.03     -- CPN実施で3%UP×確度100%  
                     WHEN F_CPN_Flag 　in ( "0.未着手","1.アプローチ", "2.アポ取得" )　THEN Velocity + 0.003   -- CPN実施で3%UP×確度10%
       ## Debut CPN期間の獲得速度の調整
                     WHEN D_CPN_Flag = '②提案中' THEN Velocity + 0.01   -- CPN実施で2%UP×確度50%    
                     WHEN D_CPN_Flag = '③合意' 　THEN Velocity + 0.02     -- CPN実施で2%UP×確度100%  
                     WHEN D_CPN_Flag　= '①未提案' 　THEN Velocity + 0.002   -- CPN実施で2%UP×確度10%
                ELSE Velocity END 
           WHEN date >= CPN_EndDate THEN Velocity 
         ELSE Velocity END AS CPN_Include_Velocity
   ## CPN時のVelocityを集計
      , CASE WHEN D_CPN_Flag = '③合意' THEN 0.025 ELSE 0.03 END AS  CPN_Velocity   -- Debut対象はSunday加味してざっくり 0.025 後は0.03と定義
      ,  seg
      ,  SUM(Sunday_Flg) OVER (Partition by TargetBrandList.BrandID ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)   -- 前日までのSUNDAYの日のカウント
         + SUM(CASE WHEN date BETWEEN CPN_StartDate AND CPN_EndDate THEN 1 ELSE 0 END) OVER (Partition by TargetBrandList.BrandID ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)　-- 前日までのCPNの日のカウント
         AS CPN_COUNT
    FROM Days, TargetBrandList  
      LEFT OUTER JOIN `merpay-tmp-exdata-jp.connect_data.20200228_CPN_Info` CPN_Master 
      ON TargetBrandList.BrandID = CPN_Master.BrandID 

''', project_id=project_id)


In [0]:
# 新規店用のParameter取得
project_id = 'kouzoh-analytics-jp-prod'

NewShops_df = pd.io.gbq.read_gbq(f'''
SELECT * FROM `mercari-analytics-jp.z_issei.20202014_NewFpayrate_3month` 
''', project_id=project_id)

NewShops_df.set_index(['Partner_Size','shops_category1'],inplace=True)
NewShops_df.head(3)

AVG_FpayRate
Partner_Size shops_category1              
Large        その他                      0.31
             ショッピング・小売                0.62
             スポーツ                     0.45

In [0]:
#index作成
index_df = df.set_index(['date','BrandID'],drop=False)
index_df.head(3)

,,date,Elapsed_Days,Sunday_Flg,Partner_Size,shops_category1,PartnerID,Partner_Name,BrandID,Brand_nm,Deploy_Date,Current_Cnt_Shops,Current_Cnt_FpayShops,Current_FpayRate,Current_Cnt_Mipay,Velocity,SecondQ_Cnt_Shops,SecondQ_Cnt_FpayShops,F_CPN_Flag,D_CPN_Flag,CPN_Include_Velocity,CPN_Velocity,seg,CPN_COUNT
date,BrandID,,,,,,,,,,,,,,,,,,,,,,,
2020-03-15,00131384-bd94-448b-abbb-b620b9ad97ef,2020-03-15,1,1,Large,飲食,72057595086936973,株式会社アトム,00131384-bd94-448b-abbb-b620b9ad97ef,カルビ大将,2019-09-25,44.0,44,1.0,0.0,0.013514,44,29,None,None,0.043514,0.03,None,NaN
2020-03-16,00131384-bd94-448b-abbb-b620b9ad97ef,2020-03-16,2,0,Large,飲食,72057595086936973,株式会社アトム,00131384-bd94-448b-abbb-b620b9ad97ef,カルビ大将,2019-09-25,44.0,44,1.0,0.0,0.013514,44,29,None,None,0.013514,0.03,None,1.0
2020-03-17,00131384-bd94-448b-abbb-b620b9ad97ef,2020-03-17,3,0,Large,飲食,72057595086936973,株式会社アトム,00131384-bd94-448b-abbb-b620b9ad97ef,カルビ大将,2019-09-25,44.0,44,1.0,0.0,0.013514,44,29,None,None,0.013514,0.03,None,1.0


In [0]:
# base_data(各Brandで1record)作成
base_df = pd.DataFrame(df.groupby(['Partner_Size','PartnerID','Partner_Name','BrandID','Brand_nm','shops_category1']).min()[['Deploy_Date','date','Current_Cnt_Shops','Current_Cnt_FpayShops']])
base_df.tail()

Deploy_Date  ... Current_Cnt_FpayShops
Partner_Size PartnerID         Partner_Name BrandID                              Brand_nm    shops_category1              ...                      
Middle       72057598308019576 東和交通株式会社     b69a43ee-e337-4313-a333-a2806920918c 東和タクシー      交通機関・サービス        2019-12-11  ...                     0
             72057598309507622 株式会社ヒバリヤ     a894e327-d5f9-400d-8121-b9c9c37e6ddb 株式会社ヒバリヤ    ショッピング・小売        2020-02-26  ...                     0
             72057598315877978 株式会社オレンヂ     5eaa14f7-05a1-46e6-add9-412e7e26aca0 株式会社オレンヂ    生活関連サービス         2019-10-25  ...                     0
             72057598323402915 加茂商事株式会社     0b7d356e-665e-4f4c-9303-3f2e6d374fb2 加茂商事株式会社    スポーツ             2019-10-25  ...                     0
             72057598323943757 株式会社クロサワ眼鏡店  f773df13-4ec8-46d3-947d-9927aa4f982f 株式会社クロサワ眼鏡店 ショッピング・小売        2020-02-20  ...                     0

[5 rows x 4 columns]

In [0]:
# 新規店のFpay率のParameter算出関数
def shopscategory_fpayrate(size, category):
  return NewShops_df.loc[(size),(category)][0]
  
shopscategory_fpayrate('Large','飲食') # test

In [0]:
 # 日付のリストを取得
date_list = index_df['date'].dt.date.unique()
date_list

array([datetime.date(2020, 3, 15), datetime.date(2020, 3, 16),
       datetime.date(2020, 3, 17), datetime.date(2020, 3, 18),
       datetime.date(2020, 3, 19), datetime.date(2020, 3, 20),
       datetime.date(2020, 3, 21), datetime.date(2020, 3, 22),
       datetime.date(2020, 3, 23), datetime.date(2020, 3, 24),
       datetime.date(2020, 3, 25), datetime.date(2020, 3, 26),
       datetime.date(2020, 3, 27), datetime.date(2020, 3, 28),
       datetime.date(2020, 3, 29), datetime.date(2020, 3, 30),
       datetime.date(2020, 3, 31)], dtype=object)

In [0]:
from datetime import timedelta,time
import datetime
## Forecastの生成

for index, item in base_df.iterrows():
  base_df.loc[index,'ontheday_fpayshops'] = item['Current_Cnt_FpayShops']   # 本日のonthedayfpayshop == 現在のFpayshop数 
  ## 既存店のロジック
  if base_df.at[index,'Deploy_Date'] <= base_df.at[index,'date']:   ## Deploydateが本日より前なら既存ロジックに当てはめる
    for t_date in date_list:
      ontheday_akimaguchi = base_df.loc[index,'Current_Cnt_Shops'] -  base_df.loc[index,'ontheday_fpayshops']  # 空き間口数 = ショップ数 - その日のFpay店舗数
      brand = index[3]
      ontheday_fpaykakutoku = ontheday_akimaguchi * index_df.xs([t_date,brand],level=['date','BrandID'])['CPN_Include_Velocity'][0].astype('float64')  # その日のfpay獲得数== その日の空き間口×velocity
      base_df.loc[index,'ontheday_fpayshops'] += ontheday_fpaykakutoku
      base_df.loc[index,t_date] = int(base_df.loc[index,'ontheday_fpayshops'])
  ## 新規店のロジック(ファミマの例外処理)
  elif index[2] == '株式会社ファミリーマート':
      for t_date in date_list:
        if t_date == datetime.date(2020, 3, 31):
          base_df.loc[index,t_date] = 3000
        else:
          base_df.loc[index,t_date] = 0
  ## 新規店のロジック
  else:
    size = index[0]       # Partner_Size取得
    category = index[5]   # category取得
    try:
      base_df.loc[index,'Avg_FpayRate'] = shopscategory_fpayrate(size,category)
      for t_date in date_list:
        delta_days = datetime.datetime.combine(t_date, time()) - base_df.loc[index,'Deploy_Date']   # 各日でdeltadaysを計算する
        if delta_days.total_seconds()/3600/24 >=0:   # deltadaysが正だったら以下のロジックを当てはめる。違ったら0にする
          base_df.loc[index,t_date] = base_df.loc[index,'Current_Cnt_Shops'] * (delta_days.total_seconds()/3600/24) * (base_df.loc[index,'Avg_FpayRate']/90)
        else:
          base_df.loc[index,t_date] = 0
    except:
      pass
        

base_df.tail()


Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID         Partner_Name BrandID                              Brand_nm    shops_category1              ...             
Middle       72057598308019576 東和交通株式会社     b69a43ee-e337-4313-a333-a2806920918c 東和タクシー      交通機関・サービス        2019-12-11  ...          NaN
             72057598309507622 株式会社ヒバリヤ     a894e327-d5f9-400d-8121-b9c9c37e6ddb 株式会社ヒバリヤ    ショッピング・小売        2020-02-26  ...          NaN
             72057598315877978 株式会社オレンヂ     5eaa14f7-05a1-46e6-add9-412e7e26aca0 株式会社オレンヂ    生活関連サービス         2019-10-25  ...          NaN
             72057598323402915 加茂商事株式会社     0b7d356e-665e-4f4c-9303-3f2e6d374fb2 加茂商事株式会社    スポーツ             2019-10-25  ...          NaN
             72057598323943757 株式会社クロサワ眼鏡店  f773df13-4ec8-46d3-947d-9927aa4f982f 株式会社クロサワ眼鏡店 ショッピング・小売        2020-02-20  ...          NaN

[5 rows x 23 columns]

In [0]:
base_df.head()

Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID          Partner_Name          BrandID            Brand_nm              shops_category1              ...             
Large        0016F0000393tcOQAQ 株式会社ファミリーマート          0016F0000393tcOQAQ 株式会社ファミリーマート          ショッピング・小売        2020-03-31  ...          NaN
             0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 飲食               2020-03-26  ...         0.57
             0016F0000396mkGQAQ 山崎製パン株式会社             0016F0000396mkGQAQ 山崎製パン株式会社             ショッピング・小売        2020-03-23  ...         0.62
             0016F0000396mxFQAQ 株式会社オートバックスセブン        0016F0000396mxFQAQ 株式会社オートバックスセブン        自動車・バイク          2020-03-31  ...         0.20
             0016F00003BifnXQAR 大東企業株式会社              0016F00003BifnXQAR 大東企業株式会社              飲食               2020-03-31  ...         0.57

[5 rows x 23 columns]

In [0]:
base_df.sum()

Current_Cnt_Shops        165141.000000
Current_Cnt_FpayShops     91672.000000
ontheday_fpayshops        98064.603416
2020-03-15                92645.000000
2020-03-16                92774.000000
2020-03-17                92901.006333
2020-03-18                93080.411667
2020-03-19                93254.817000
2020-03-20                93430.222333
2020-03-21                93604.924333
2020-03-22                95018.363444
2020-03-23                95164.802556
2020-03-24                95331.525667
2020-03-25                95481.248778
2020-03-26                95642.971889
2020-03-27                95800.025444
2020-03-28                95958.079000
2020-03-29                97354.132556
2020-03-30                97442.186111
2020-03-31               100523.239667
Avg_FpayRate                 23.810000
dtype: float64

In [0]:
# spreadsheet 操作用ライブラリ
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
auth.authenticate_user()

# Access to Google Spread Sheet
gc = gspread.authorize(GoogleCredentials.get_application_default())
# Open spreadsheet by key
wb = gc.open_by_key('1q5Ng5xX_x8TVj9160iluz3ph0LPG96X1SUk7T_ydeok')
# https://tanuhack.com/library-gspread/ : gspreadの使い方

In [0]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe


update_sheet = wb.worksheet("forecasttest")

# 作ったDataFrameを貼り付ける。
set_with_dataframe(update_sheet, base_df.reset_index(),resize=False, 
include_index=False)

# gspread_dataframe 参考：　https://teratail.com/questions/137584